### SageMaker  Endpoint 部署ChatYuan
  
[ChatYuan](https://huggingface.co/ClueAI/ChatYuan-large-v1): 元语功能型对话大模型 , 这个模型可以用于问答、结合上下文做对话、做各种生成任务，包括创意性写作，也能回答一些像法律、新冠等领域问题。它基于PromptCLUE-large结合数亿条功能对话多轮对话数据进一步训练得到。

PromptCLUE-large在1000亿token中文语料上预训练，累计学习1.5万亿中文token，并且在数百种任务上进行Prompt任务式训练。针对理解类任务，如分类、情感分析、抽取等，可以自定义标签体系；针对多种生成任务，可以进行采样自由生成。

#### 准备
1. 升级boto3, sagemaker python sdk  
2. 准备inference.py, requirements.txt

In [ ]:
!pip install --upgrade boto3
!pip install --upgrade sagemaker

In [ ]:
import boto3
import sagemaker

account_id = boto3.client('sts').get_caller_identity().get('Account')
region_name = boto3.session.Session().region_name

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()

print(role)
print(bucket)

In [ ]:
!touch dummy
!tar czvf model.tar.gz dummy
assets_dir = 's3://{0}/{1}/assets/'.format(bucket, 'chatyuan')
model_data = 's3://{0}/{1}/assets/model.tar.gz'.format(bucket, 'chatyuan')
!aws s3 cp model.tar.gz $assets_dir
!rm -f dummy model.tar.gz

### 设置模型推理参数
1. model_name: Huggingface diffusers models (not support single check point format)
2. model_args: diffuser StableDiffusionPipeline init arguments

In [ ]:
model_name = None
entry_point = 'inference.py'
framework_version = '1.12'
py_version = 'py38'
bucket='m.flowq.io'
model_environment = {
    'SAGEMAKER_MODEL_SERVER_TIMEOUT':'600', 
    'SAGEMAKER_MODEL_SERVER_WORKERS': '1', 
}

In [ ]:
from sagemaker.pytorch.model import PyTorchModel

model = PyTorchModel(
    name = model_name,
    model_data = model_data,
    entry_point = 'inference.py',
    source_dir = './code',
    role = role,
    framework_version = framework_version, 
    py_version = py_version,
    env = model_environment
)

### 部署 SageMaker Endpoint
部署推理服务

In [ ]:
endpoint_name = None
instance_type = 'ml.g4dn.xlarge'
instance_count = 1

from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
predictor = model.deploy(
    endpoint_name = endpoint_name,
    instance_type = instance_type, 
    initial_instance_count = instance_count,
    serializer = JSONSerializer(),
    deserializer = JSONDeserializer()
)

### ChatYuan 测试


In [ ]:
inputs= {
    "ask": "费德勒网球水平怎么样? "

}

response = predictor.predict(inputs)
print(response["answer"])


In [ ]:
#我们来查看一下刚部署的这个chatyuan模型对应的SageMaker inference endpoint
your_chatyuan_endpoint_name = predictor.endpoint_name
your_chatyuan_endpoint_name

利用已经在SageMaker real time inference endpoint部署的ChatYuan LLM模型来模拟单轮对话和多轮对话。

下面的代码建议在SageMaker Notebook上来运行。

In [ ]:
import json
import boto3

client = boto3.client('runtime.sagemaker')

def query_endpoint_with_json_payload(encoded_json):
    response = client.invoke_endpoint(EndpointName=your_chatyuan_endpoint_name, ContentType='application/json', Body=encoded_json)
    return response

def parse_response_texts(query_response):
    model_predictions = json.loads(query_response['Body'].read())
    generated_text = model_predictions["answer"]
    return generated_text


先简单测试一下chatyuan针对单个问题的回答

In [ ]:
payload = {"ask": "信息抽取：\n2022年世界杯的冠军是阿根廷队伍，梅西是MVP\n问题：国家名，人名\n答案："}
query_response = query_endpoint_with_json_payload(json.dumps(payload).encode('utf-8'))
generated_texts = parse_response_texts(query_response)
print(generated_texts)

单轮对话：每个问题/query都是独立的，问题之间没有关联性。

In [ ]:
#1.首先需要一个简单的开场拍。
print("用户：你好！\n小元：您好!我是元语AI。我可以回答您的问题、写文章、写作业、翻译，对于一些法律等领域的问题我也可以给你提供信息。")

#2.在同一个session中持续对话，但是每次对话之间没有什么关联。
while True:
    command = input("用户：")
    if command == 'quit':
        break;
    
    #print(command)
    payload = {"ask": "\n用户："+ command + "\n"}
    #print(payload["ask"])
    query_response = query_endpoint_with_json_payload(json.dumps(payload).encode('utf-8'))
    generated_texts = "小元：" + parse_response_texts(query_response)
    print(generated_texts)

多轮对话模拟：我们这里来测试一下chatyuan的多轮对话能力。

In [ ]:
#1.首先需要开场拍来引导chatyuan，其实就是给它一个上下文来启动所谓的对话session。
payload = {"ask": "用户：你好！\n小元：您好!我是元语AI。我可以回答您的问题、写文章、写作业、翻译，对于一些法律等领域的问题我也可以给你提供信息。"}
print(payload["ask"])
generated_texts = ""

#在这里简单模拟多轮对话时，发送给SageMaker endpoint的payload会越来越大，这里对payload大致做一个限制。
session_len = 10 * 1024 * 1024 

#2.在同一个session中持续对话，为了有多轮对话的效果，把每一轮的信息(问题-回答对)都带上来告诉chatyuan之前的上下文。
while True:
    command = input("用户：")
    if command == 'quit':
        break;
    
    #print(command)
    whole_context = payload["ask"] + generated_texts + "\n用户："+ command + "\n"
    payload = {"ask": whole_context}
    if len(whole_context) > session_len:
        print("上下文信息太多了，当前对话session要退出了！")
        break;
    #print(payload["ask"])
    query_response = query_endpoint_with_json_payload(json.dumps(payload).encode('utf-8'))
    generated_texts = "小元：" + parse_response_texts(query_response)
    print(generated_texts)

### 删除SageMaker  Endpoint
删除推理服务

In [ ]:
predictor.delete_endpoint()